In [1]:
# %%capture
!pip install alibi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.1/522.1 kB 10.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 39.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 27.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 38.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 10.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 16.0 MB/s eta 

In [2]:
# %%capture
!pip install alibi-detect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.5/381.5 kB 6.7 MB/s eta 0:00:00:00:01


In [3]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import scipy.stats as stats
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle 

import alibi 
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.saving import save_detector, load_detector

import warnings
warnings.filterwarnings('ignore')

Matplotlib is building the font cache; this may take a moment.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/cuda/__init__.py:616: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
# Loading a sample dataset
data_raw = pd.read_csv('loan_data_set.csv')
data_raw = data_raw.drop(['Credit_History','Loan_ID'],axis=1) # dropping this feature because it is high importance and this will hide us from detecting drift in other variables
print(data_raw.shape)
data_raw.head()

(614, 11)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,Urban,Y


In [6]:
data_raw.isna().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Property_Area         0
Loan_Status           0
dtype: int64

In [7]:
data_raw = data_raw.dropna()

In [8]:
data_raw.isna().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Property_Area        0
Loan_Status          0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_raw = data_raw.drop('Loan_Status',axis=1)
Y_raw = data_raw[['Loan_Status']]

x_train, x_test, y_train, y_test = train_test_split(X_raw,Y_raw,stratify=Y_raw,test_size=0.3)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(366, 10) (366, 1)
(157, 10) (157, 1)


In [10]:
# Create the dictionary mapping for categorical features to numbers for the DataDrift library to work
cat_features = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']

temp = x_train.copy()

for cat_var in cat_features:
    fileName = cat_var+"_mapping.pkl"
    temp[cat_var] = temp[cat_var].astype('category')
    obj = dict(zip(temp[cat_var],temp[cat_var].cat.codes))
    print(obj)
    with open(fileName,'wb') as f:
        pickle.dump(obj,f)

{'Male': 1, 'Female': 0}
{'Yes': 1, 'No': 0}
{'3+': 3, '0': 0, '1': 1, '2': 2}
{'Graduate': 0, 'Not Graduate': 1}
{'No': 0, 'Yes': 1}
{'Semiurban': 1, 'Urban': 2, 'Rural': 0}


In [11]:
# Encode the categorical features to numbers for the DataDrift library to work
cat_features = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']

for cat_var in cat_features:
    fileName = cat_var+"_mapping.pkl"
    with open(fileName,'rb') as f:
        obj = pickle.load(f)

    x_train[cat_var] = x_train[cat_var].map(obj)

In [12]:
print(x_train.shape)
x_train.head()

(366, 10)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area
359,1,1,3,0,0,5167,3167.0,200.0,360.0,1
490,1,0,0,1,0,2699,2785.0,96.0,360.0,1
254,1,0,0,0,1,16250,0.0,192.0,360.0,2
511,1,1,1,0,0,6065,2004.0,250.0,360.0,1
575,1,1,2,0,0,3159,461.0,108.0,84.0,2


In [13]:
print(y_train.shape)
y_train.head()

(366, 1)


,Loan_Status
359,Y
490,Y
254,N
511,Y
575,Y


In [15]:
# Splitting the data to a REFERENCE_SET and two TEST_SETs
n_ref = 200
n_test = 60

X = x_train.copy()
Y = y_train.copy()

X_ref, X_t0, X_t1 = X.iloc[:n_ref], X.iloc[n_ref:n_ref + n_test], X.iloc[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

((200, 10), (60, 10), (60, 10))

##### Detect Data Drift:

We need to provide the drift detector with the columns which contain categorical features so it knows which features require the Chi-Squared and which ones require the K-S univariate test. We can either provide a dict with as keys the column indices and as values the number of possible categories or just set the values to None and let the detector infer the number of categories from the reference data as in the example below:

In [16]:
X_ref.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area
359,1,1,3,0,0,5167,3167.0,200.0,360.0,1
490,1,0,0,1,0,2699,2785.0,96.0,360.0,1
254,1,0,0,0,1,16250,0.0,192.0,360.0,2
511,1,1,1,0,0,6065,2004.0,250.0,360.0,1
575,1,1,2,0,0,3159,461.0,108.0,84.0,2


In [17]:
categories_per_feature = {f: None for f in [0,1,2,3,4,9]}
categories_per_feature

{0: None, 1: None, 2: None, 3: None, 4: None, 9: None}

In [18]:
# Initialize the detector
cd = TabularDrift(X_ref.values, p_val=.05, categories_per_feature=categories_per_feature)

In [19]:
# # We can also save/load an initialised detector: (not working; throwing some error)
# filepath = 'TrainedTabularDriftObj'  # change to directory where detector is saved
# save_detector(cd, filepath)
# trained_drift_model = load_detector(filepath,allow_pickle=True)

In [20]:
# Using pickle to save and load it the trained detector
with open('Trained_Drift_Detector.pkl','wb') as F:
    pickle.dump(cd,F)

with open('Trained_Drift_Detector.pkl','rb') as F:
    trained_drift_model = pickle.load(F)    

**Predicting on the Test data:**

In [22]:
preds = trained_drift_model.predict(X_t0.values)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? No!


In [23]:
preds

{'data': {'is_drift': 0,
  'distance': array([2.0155038e-03, 7.6282811e-01, 3.3587949e+00, 8.4635419e-01,
         8.1489675e-02, 1.0166667e-01, 9.6666664e-02, 8.8333331e-02,
         2.5000000e-02, 2.2018805e+00], dtype=float32),
  'p_val': array([0.9641915 , 0.38244492, 0.33954296, 0.3575858 , 0.7752887 ,
         0.6905309 , 0.7469461 , 0.83453274, 1.        , 0.33255827],
        dtype=float32),
  'threshold': 0.005},
 'meta': {'name': 'TabularDrift',
  'online': False,
  'data_type': None,
  'version': '0.12.0',
  'detector_type': 'drift'}}

In [24]:
# For chi-Square demonstration
cat_features = ['Gender','Married','Education','Self_Employed','Property_Area']

for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = X_ref.columns.tolist()[f]
    # print(f, fname)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

Gender -- Chi2 0.002 -- p-value 0.964
Married -- Chi2 0.763 -- p-value 0.382
Dependents -- Chi2 3.359 -- p-value 0.340
Education -- Chi2 0.846 -- p-value 0.358
Self_Employed -- Chi2 0.081 -- p-value 0.775
ApplicantIncome -- K-S 0.102 -- p-value 0.691
CoapplicantIncome -- K-S 0.097 -- p-value 0.747
LoanAmount -- K-S 0.088 -- p-value 0.835
Loan_Amount_Term -- K-S 0.025 -- p-value 1.000
Property_Area -- Chi2 2.202 -- p-value 0.333


Testing whether changing few values in one of the column have any impact or not for the same dataset

In [25]:
test = X_t0.copy()
test.iloc[:20,0] = 5
print(test.shape)
test.head()

(60, 10)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area
152,5,0,0,0,0,9166,0.0,244.0,360.0,2
192,5,1,0,1,0,6033,0.0,160.0,360.0,2
238,5,0,1,0,0,3812,0.0,112.0,360.0,0
69,5,0,0,0,0,4300,0.0,136.0,360.0,1
156,5,1,1,0,0,6000,0.0,160.0,360.0,0


In [26]:
preds = trained_drift_model.predict(test.values)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

Drift? Yes!


In [27]:
preds

{'data': {'is_drift': 1,
  'distance': array([7.2297333e+01, 7.6282811e-01, 3.3587949e+00, 8.4635419e-01,
         8.1489675e-02, 1.0166667e-01, 9.6666664e-02, 8.8333331e-02,
         2.5000000e-02, 2.2018805e+00], dtype=float32),
  'p_val': array([1.9990954e-16, 3.8244492e-01, 3.3954296e-01, 3.5758579e-01,
         7.7528870e-01, 6.9053090e-01, 7.4694610e-01, 8.3453274e-01,
         1.0000000e+00, 3.3255827e-01], dtype=float32),
  'threshold': 0.005},
 'meta': {'name': 'TabularDrift',
  'online': False,
  'data_type': None,
  'version': '0.12.0',
  'detector_type': 'drift'}}

In [28]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = X_ref.columns.tolist()[f]
    # print(f, fname)
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

Gender -- Chi2 72.297 -- p-value 0.000
Married -- Chi2 0.763 -- p-value 0.382
Dependents -- Chi2 3.359 -- p-value 0.340
Education -- Chi2 0.846 -- p-value 0.358
Self_Employed -- Chi2 0.081 -- p-value 0.775
ApplicantIncome -- K-S 0.102 -- p-value 0.691
CoapplicantIncome -- K-S 0.097 -- p-value 0.747
LoanAmount -- K-S 0.088 -- p-value 0.835
Loan_Amount_Term -- K-S 0.025 -- p-value 1.000
Property_Area -- Chi2 2.202 -- p-value 0.333


The above method will give us an indicator for the whole dataset and drift will be 1 or 0 based on all the features in the dataset.


The only difference between the above and the below is that 'is_drift' key in the below has drift indicator for each feature whereas in the above it will be a single 1/0 for the whole dataset.

In [29]:
# If you are interested in individual feature-wise drift, this is also possible:
fpreds = trained_drift_model.predict(test.values, drift_type='feature')
fpreds

{'data': {'is_drift': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'distance': array([7.2297333e+01, 7.6282811e-01, 3.3587949e+00, 8.4635419e-01,
         8.1489675e-02, 1.0166667e-01, 9.6666664e-02, 8.8333331e-02,
         2.5000000e-02, 2.2018805e+00], dtype=float32),
  'p_val': array([1.9990954e-16, 3.8244492e-01, 3.3954296e-01, 3.5758579e-01,
         7.7528870e-01, 6.9053090e-01, 7.4694610e-01, 8.3453274e-01,
         1.0000000e+00, 3.3255827e-01], dtype=float32),
  'threshold': 0.05},
 'meta': {'name': 'TabularDrift',
  'online': False,
  'data_type': None,
  'version': '0.12.0',
  'detector_type': 'drift'}}

In [30]:
for f in range(trained_drift_model.n_features):
    stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    # print(f, stat)
    fname = X_ref.columns.tolist()[f]
    # print(f, fname)
    is_drift = fpreds['data']['is_drift'][f]
    stat_val, p_val = fpreds['data']['distance'][f], fpreds['data']['p_val'][f]
    print(f'{fname} -- Drift? {labels[is_drift]} -- {stat} {stat_val:.3f} -- p-value {p_val:.3f}')

Gender -- Drift? Yes! -- Chi2 72.297 -- p-value 0.000
Married -- Drift? No! -- Chi2 0.763 -- p-value 0.382
Dependents -- Drift? No! -- Chi2 3.359 -- p-value 0.340
Education -- Drift? No! -- Chi2 0.846 -- p-value 0.358
Self_Employed -- Drift? No! -- Chi2 0.081 -- p-value 0.775
ApplicantIncome -- Drift? No! -- K-S 0.102 -- p-value 0.691
CoapplicantIncome -- Drift? No! -- K-S 0.097 -- p-value 0.747
LoanAmount -- Drift? No! -- K-S 0.088 -- p-value 0.835
Loan_Amount_Term -- Drift? No! -- K-S 0.025 -- p-value 1.000
Property_Area -- Drift? No! -- Chi2 2.202 -- p-value 0.333


##### Detect Model Drift:

In [31]:
# Evidently can be used to check the model drift
# We can calculate the performance metrics for the model and pickle it.
#Then at the time of monitoring check for how much deviation the scoring data is from the actual obsered metrics.
#if the deviation is more than +-10% (or any threshold) then flag it as model drift
# and proceed for retraining (same process used in evidently library)

In [32]:
x_train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area
359,1,1,3,0,0,5167,3167.0,200.0,360.0,1
490,1,0,0,1,0,2699,2785.0,96.0,360.0,1
254,1,0,0,0,1,16250,0.0,192.0,360.0,2
511,1,1,1,0,0,6065,2004.0,250.0,360.0,1
575,1,1,2,0,0,3159,461.0,108.0,84.0,2


In [33]:
# The flow I followed:

# Train a model on the training data and save the model metrics & the model
# Then use the categorical mapping file to prepare the test data and score it using the trained model
# compare the train and test performance (also their deviation)
# Then change the test data a bit and then do the above two steps again.
# wrap the above steps as a function that will return model drift or not based on train and test performance deviation

In [36]:
# RF
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 

rf = RandomForestClassifier(max_depth=8, min_samples_split=10)
rf.fit(x_train,y_train)
rf.score(x_train,y_train)

0.8005464480874317

In [38]:
y_train_encode = [1 if i == 'Y' else 0 for i in y_train.values]
y_pred = rf.predict_proba(x_train)[:,1]
y_pred_class = rf.predict(x_train)
y_pred_class_encode = [1 if i == 'Y' else 0 for i in y_pred_class]
print(metrics.classification_report(y_train,y_pred_class))
#metrics.confusion_matrix(rf,x_train,y_train)


              precision    recall  f1-score   support

           N       1.00      0.33      0.50       109
           Y       0.78      1.00      0.88       257

    accuracy                           0.80       366
   macro avg       0.89      0.67      0.69       366
weighted avg       0.84      0.80      0.76       366



In [39]:
# logging the training performance metrics and the trained model
precision = metrics.precision_score(y_train_encode,y_pred_class_encode)
recall = metrics.recall_score(y_train_encode,y_pred_class_encode)
roc_auc = metrics.roc_auc_score(y_train_encode,y_pred)

training_performance_metrics = dict()
training_performance_metrics['Precision'] = np.round(precision,2)
training_performance_metrics['Recall'] = np.round(recall,2)
training_performance_metrics['Roc-Auc'] = np.round(roc_auc,2)

print(training_performance_metrics)

with open('Training_Perfrom_Metrics.pkl','wb') as F:
    pickle.dump(training_performance_metrics,F)

with open('RF_Loan_Model.pkl','wb') as F:
    pickle.dump(rf,F)


{'Precision': 0.78, 'Recall': 1.0, 'Roc-Auc': 0.95}


In [43]:
# Function to check model drift
def check_model_drift(ref_metric_dict,cur_metric_dict,type='classification',tol=0.1):
    if type == 'classification':
        precision_change = abs((cur_metric_dict['Precision']-ref_metric_dict['Precision'])/ref_metric_dict['Precision'])
        recall_change = abs((cur_metric_dict['Recall']-ref_metric_dict['Recall'])/ref_metric_dict['Recall'])
        roc_auc_change = abs((cur_metric_dict['Roc-Auc']-ref_metric_dict['Roc-Auc'])/ref_metric_dict['Roc-Auc'])

        counter = 0
        for i in [precision_change,recall_change,roc_auc_change]:
            if i > 0.1:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            print("Change in Precision: "+ str(np.round(100*precision_change,2))+"%")
            print("Change in Recall: "+ str(np.round(100*recall_change,2))+"%")
            print("Change in Roc-Auc: "+ str(np.round(100*roc_auc_change,2))+"%")
            return 1
        else:
            print("There is no model drift.")
            return 0

    else:
        pass 

In [42]:
test = training_performance_metrics.copy()
test['Precision'] = 0.69
test

{'Precision': 0.69, 'Recall': 1.0, 'Roc-Auc': 0.95}

In [44]:
check_model_drift(training_performance_metrics,test)

ALERT! There is a model drift.
Change in Precision: 11.54%
Change in Recall: 0.0%
Change in Roc-Auc: 0.0%


1

Applying the preprocessing and preparing the test data for scoring and model drift checking

In [45]:
print(x_test.shape)
x_test.head()

(157, 10)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area
474,Male,No,2,Graduate,No,5532,4648.0,162.0,360.0,Rural
568,Female,No,0,Graduate,No,2378,0.0,9.0,360.0,Urban
16,Male,No,1,Not Graduate,No,3596,0.0,100.0,240.0,Urban
42,Male,Yes,0,Graduate,No,2400,0.0,75.0,360.0,Urban
522,Male,Yes,3+,Graduate,Yes,5677,1424.0,100.0,360.0,Rural


In [46]:
x_test_with_noise = x_test.copy()

In [47]:
# Adding noise to the data
x_test_with_noise.iloc[100:110,0] = 'None'
x_test_with_noise.iloc[120:130,0] = 'None'

x_test_with_noise.iloc[100:110,7] = 1500
x_test_with_noise.iloc[120:130,7] = 1600

x_test_with_noise.iloc[90:110,6] = 7000
x_test_with_noise.iloc[120:130,6] = 7000

In [48]:
x_test_with_noise.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area
474,Male,No,2,Graduate,No,5532,4648.0,162.0,360.0,Rural
568,Female,No,0,Graduate,No,2378,0.0,9.0,360.0,Urban
16,Male,No,1,Not Graduate,No,3596,0.0,100.0,240.0,Urban
42,Male,Yes,0,Graduate,No,2400,0.0,75.0,360.0,Urban
522,Male,Yes,3+,Graduate,Yes,5677,1424.0,100.0,360.0,Rural


In [49]:
def prepare_data(x_t):
    x_t = x_t.dropna()
    cat_features = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']

    for cat_var in cat_features:
        fileName = cat_var+"_mapping.pkl"
        with open(fileName,'rb') as f:
            obj = pickle.load(f)

        # Checking if any new categories in the columns and encoding them with -99
        ref_cats = list(obj.keys())
        cur_cats = x_t[cat_var].unique().tolist()

        unseen_cats = list(set(cur_cats).difference(set(ref_cats)))
        for cat in unseen_cats:
            if cat not in obj.keys():
                obj[cat] = -99
        x_t[cat_var] = x_t[cat_var].map(obj)
    
    return x_t 

In [50]:
# temp = x_test.copy()
temp = x_test_with_noise.copy()
x_test_prepared = prepare_data(temp)
print(x_test_prepared.shape)
x_test_prepared.head()

(157, 10)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area
474,1,0,2,0,0,5532,4648.0,162.0,360.0,0
568,0,0,0,0,0,2378,0.0,9.0,360.0,2
16,1,0,1,1,0,3596,0.0,100.0,240.0,2
42,1,1,0,0,0,2400,0.0,75.0,360.0,2
522,1,1,3,0,1,5677,1424.0,100.0,360.0,0


In [51]:
x_test_prepared.Gender.value_counts()

 1     108
 0      29
-99     20
Name: Gender, dtype: int64

In [52]:
# Create a simulation that predicts every row and after 50 rows gets the performance metrics for the same and compare it with reference metrics for model drift
x_test_raw = x_test_prepared.copy()
y_test_raw = y_test.copy()

for i in range(x_test_prepared.shape[0]): # x_test_prepared.shape[0]
    temp = x_test_prepared.iloc[i:i+1]    
    
    # loading the trained model
    with open('RF_Loan_Model.pkl','rb') as F:
        model = pickle.load(F)
    
    # Loading the Training performance metrics
    with open('Training_Perfrom_Metrics.pkl', 'rb') as F:
        train_performance_metrics = pickle.load(F)
    
    # Getting predictions on the test data
    temp['prediction_prob'] = model.predict_proba(x_test_prepared.iloc[i:i+1])[:,1]
    temp['prediction_class'] = model.predict(x_test_prepared.iloc[i:i+1])
    # display(temp)

    # Concatenating the dataframe
    if i == 0:
        df_log = temp.copy()
    else:
        df_log = pd.concat([df_log,temp],axis=0)

    # After every 50 predictions get the performance metrics and check for model drift
    if i % 50 == 0 and i != 0:
        print(i)
        y_temp = y_test.iloc[:i+1,:]
        y_test_encode = [1 if i == 'Y' else 0 for i in y_temp.values]
        y_pred = df_log['prediction_prob'].values
        y_pred_class = df_log['prediction_class'].values
        y_pred_class_encode = [1 if i == 'Y' else 0 for i in y_pred_class]
        print(y_temp.shape)
        print(y_pred_class.shape)
        print(metrics.classification_report(y_temp,y_pred_class))
        #metrics.plot_confusion_matrix(rf,x_test_prepared.iloc[:i+1,:],y_temp)

        precision = metrics.precision_score(y_test_encode,y_pred_class_encode)
        recall = metrics.recall_score(y_test_encode,y_pred_class_encode)
        roc_auc = metrics.roc_auc_score(y_test_encode,y_pred)

        ref_perform_metric = dict()
        ref_perform_metric['Precision'] = precision
        ref_perform_metric['Recall'] = recall
        ref_perform_metric['Roc-Auc'] = roc_auc

        check_model_drift(train_performance_metrics,ref_perform_metric)


50
(51, 1)
(51,)
              precision    recall  f1-score   support

           N       0.50      0.14      0.22        14
           Y       0.74      0.95      0.83        37

    accuracy                           0.73        51
   macro avg       0.62      0.54      0.53        51
weighted avg       0.68      0.73      0.67        51

ALERT! There is a model drift.
Change in Precision: 4.53%
Change in Recall: 5.41%
Change in Roc-Auc: 26.64%
100
(101, 1)
(101,)
              precision    recall  f1-score   support

           N       0.50      0.12      0.19        34
           Y       0.68      0.94      0.79        67

    accuracy                           0.66       101
   macro avg       0.59      0.53      0.49       101
weighted avg       0.62      0.66      0.59       101

ALERT! There is a model drift.
Change in Precision: 13.15%
Change in Recall: 5.97%
Change in Roc-Auc: 31.89%
150
(151, 1)
(151,)
              precision    recall  f1-score   support

           N     

In [53]:
print(df_log.shape)
df_log.head()

(157, 12)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area,prediction_prob,prediction_class
474,1,0,2,0,0,5532,4648.0,162.0,360.0,0,0.737011,Y
568,0,0,0,0,0,2378,0.0,9.0,360.0,2,0.756867,Y
16,1,0,1,1,0,3596,0.0,100.0,240.0,2,0.681611,Y
42,1,1,0,0,0,2400,0.0,75.0,360.0,2,0.630195,Y
522,1,1,3,0,1,5677,1424.0,100.0,360.0,0,0.687559,Y
